# CARS prepare and compute_dsm pipelines
This notebook shows how to configure and launch CARS prepare and compute_dsm pipeline. 
This pipeline takes input configuration with two captors images and their geometric models to generate prepare inputs to the main CARS compute_dsm pipeline.

BEWARE: The notebooks are not made to run big production : keep input data images low and not too much pairs in pipelines loops. 


In [ ]:
# Load cars_generic : generic notebook configuration and load CARS import. 
%run cars_generic.ipynb

# Generic configuration that can be changed
print("\nCARS Configuration directories:")

# CARS source root directory 
#CARS_ROOT_DIR="TO CHANGE" # Default in cars_generic: default parent notebook directory (..)
print("CARS_ROOT_DIR: {}".format(CARS_ROOT_DIR))

# CARS working directory
#CARS_WORK_DIR="TO CHANGE" # Default in cars_generic: $TMPDIR/cars-work
print("CARS_WORK_DIR: {}".format(CARS_WORK_DIR))

# CARS inputs directory with data and inputs json prepare configuration
#CARS_INPUTS_DIR="TO CHANGE" #Default: "data_samples" copied from CARS demo source in cars_generic 
print("CARS_INPUTS_DIR: {} ".format(CARS_INPUTS_DIR))

In [ ]:
## Import CARS prepare

In [ ]:
# Import cars read conf functions
from cars.conf.input_parameters import read_input_parameters  # for prepare
from cars.conf.output_prepare import read_preprocessing_content_file # for compute_dsm

# Import correlator configuration 
from cars.plugins.matching.correlator_configuration import corr_conf

# Import pipelines
from cars.pipelines import prepare, compute_dsm

## Check and Get Inputs data
Input data can be changed manually but you need to create your own input directory with data and adequate input json configuration. Then, change the CARS_INPUT_DIR above.  

Show inputs directory containing data and JSON configuration. 

In [ ]:
!ls -al {CARS_INPUTS_DIR}

In [ ]:
# Get all json files in inputs_directory recursively : Assuming all json files are CARS input files !
list_prepare_input_file_path = glob.glob(CARS_INPUTS_DIR+"/*.json")
list_prepare_input_file_path

Show CARS Json input configuration files: 
The configuration needs at least: 
- img1 : left stereo image (with geometric models inside or in geom file with same name)
- img2 : right stereo image (with geometric models inside or in geom file with same name)
- color1: image to generate a True Ortho in parallel of the DSM


In [ ]:
list_prepare_input_file=[]
for prepare_input_file_path in list_prepare_input_file_path: 
    print("Input prepare json file: {}".format(prepare_input_file_path))
    # Get and Check json input file content
    prepare_input_file=read_input_parameters(prepare_input_file_path)
    list_prepare_input_file.append(prepare_input_file)
    print(json.dumps(prepare_input_file, indent=2))

## Prepare configuration
This part is to configure prepare pipeline. 
By default, it uses CARS demo data from source code copied in WORK_DIR/demo/ 


In [ ]:
# Create outputs_prepare subdirectory in CARS_WORK_DIR 
CARS_OUTPUTS_PREPARE_DIR = mkdir(CARS_WORK_DIR, "outputs_prepare")
print("CARS_OUTPUTS_PREPARE_DIR : {} ".format(CARS_OUTPUTS_PREPARE_DIR))

# CARS Configuration

CARS_MODE = "local_dask" # Can be also pbs_dask if you have a PBS Cluster

CARS_NB_WORKERS = 4      

CARS_WALLTIME = "01:00:00" # 1 hour for DASK

## Run CARS prepare pipeline for each pair
for each JSON pair configuration in list_prepare_input_file above

In [ ]:
# Lancement de l'étape de prepare pour chaque paire
prepare_pair_num = 1
for prepare_input_file in list_prepare_input_file: 
    # Create numbered output prepare subdirectory for each pair 
    cars_output_prepare_dir = mkdir(CARS_OUTPUTS_PREPARE_DIR, "cars_prepare-"+str(prepare_pair_num ))
    # CARS RUN PREPARE pipeline run 
    prepare.run(prepare_input_file, cars_output_prepare_dir, mode = CARS_MODE, nb_workers = CARS_NB_WORKERS, walltime=CARS_WALLTIME)
    prepare_pair_num = prepare_pair_num + 1

## CARS compute_dsm configuration

In [ ]:
# CARS Output DSM directory creation
CARS_OUTPUT_DSM_DIR = mkdir(CARS_WORK_DIR, "output_dsm")
print("CARS_OUTPUT_DSM_DIR : {} ".format(CARS_OUTPUT_DSM_DIR))

In [ ]:
# Get all content.json configuration files from cars prepare steps for input to compute_dsm
list_compute_input_file_path = glob.glob(CARS_OUTPUTS_PREPARE_DIR+'/**/content.json', recursive=True)
list_compute_input_file_path

In [ ]:
list_compute_input_file = [read_preprocessing_content_file(file_path) for file_path in list_compute_input_file_path]

In [ ]:
# Generate correlator configuration
corr_config = corr_conf.configure_correlator()

## Run CARS compute_dsm pipeline for all pairs


In [ ]:
compute_dsm.run(list_compute_input_file, CARS_OUTPUT_DSM_DIR, mode=CARS_MODE, nb_workers=CARS_NB_WORKERS,  corr_config = corr_config)

## Show DSM and color

In [ ]:
# Showing Imports
import rasterio
from rasterio.plot import show
from matplotlib import pyplot as plt
from matplotlib.colors import LightSource

In [ ]:
# Open DSM and show content with Rasterio
CARS_OUTPUT_DSM=os.path.join(CARS_OUTPUT_DSM_DIR+"/dsm.tif")
CARS_OUTPUT_COLOR=os.path.join(CARS_OUTPUT_DSM_DIR+"/clr.tif")

dsm = rasterio.open(CARS_OUTPUT_DSM)
print("DSM characteristics of {}:".format(CARS_OUTPUT_DSM))
print("Number of channels: {}".format(dsm.count))
print("Height, width = [{},{}]".format(dsm.height, dsm.width))
print("CRS: {}".format(dsm.crs))
print("Bounds: {}".format(dsm.bounds))

color = rasterio.open(CARS_OUTPUT_COLOR)
print("\nColor characteristics of {}:".format(CARS_OUTPUT_DSM))
print("Number of channels: {}".format(color.count))
print("Height, width = [{},{}]".format(color.height, color.width))
print("CRS: {}".format(color.crs))

In [ ]:
# Rasterio plot
show(dsm)

show(color)

In [ ]:
color_array =  color.read(1)
plt.imshow(color_array)
plt.show()

dsm_array = dsm.read(1)
plt.imshow(dsm_array,cmap="terrain")
plt.show()